<a href="https://colab.research.google.com/github/juliuserbach/Semantic-Features/blob/master/Semantic_Features_3DVision_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Our mapping pipeline is of the following structure:

1.   Detection of objects of certain object classes (e.g. traffic sign). Output: object bounding boxes
2.   Triangulation of objects. Output: object position relative to pose
3.   Creating map of objects. (And refining with filter, BA, etc.) Output: List of objects and their corresponding positions
4.   Visualizing map.





We start by loading the benchmark dataset. The Cityscapes dataset is used. 

Scripts for analyzing the dataset can be found here: https://github.com/mcordts/cityscapesScripts

How to download the zip files directly: https://towardsdatascience.com/download-city-scapes-dataset-with-script-3061f87b20d7



Download CityScape files: 

*   leftImg8bit_trainextra.zip
*   disparity_trainextra.zip
*   camera_trainextra.zip
*   vehicle_trainextra.zip

The files are unzipped into data/... respectively.









In [1]:
# Download and unzip (run only once!).
PATH_TO_LEFT_IMAGES = "leftImg8bit/train_extra/schweinfurt/"
PATH_TO_DISPARITY = "disparity/train_extra/schweinfurt/"
PATH_TO_CAMERA = "camera/train_extra/schweinfurt/"
PATH_TO_VEHICLE = "vehicle/train_extra/schweinfurt/"

# Remove data directory if it is already loaded (if needed).
#!rm -r data
# Login.
!wget --keep-session-cookies --save-cookies=cookies.txt --post-data 'username=ftaubner@ethz.ch&password=semantic_dudes&submit=Login' https://www.cityscapes-dataset.com/login/
# Get left camera images. I put it on OneDrive so that only Schweinfurt has to be downloaded.
!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%2199734&authkey=AC5K24PFcrSPFl4" -O leftImg8bit_trainextra.zip
# Alternatively, it can be downloaded from CityScapes website directly:
#   (!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=4)
# Extract.
!mkdir data
!unzip -qq leftImg8bit_trainextra.zip "$PATH_TO_LEFT_IMAGES"* -d data
# And delete.
!rm leftImg8bit_trainextra.zip

# Get disparity maps.
!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%2199735&authkey=AEwGRlH_TySnyPM" -O disparity_trainextra.zip
# Original file:
#   (!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=22)
!unzip -qq disparity_trainextra.zip "$PATH_TO_DISPARITY"* -d data
!rm disparity_trainextra.zip
# Get camera intrinsics. 
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=9
!unzip -qq camera_trainextra.zip "$PATH_TO_CAMERA"* -d data
!rm camera_trainextra.zip
# Get vehicle odometry.
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=11
!unzip -qq vehicle_trainextra.zip "$PATH_TO_VEHICLE"* -d data
!rm vehicle_trainextra.zip

import os
PATH_TO_LEFT_IMAGES = os.path.join("data/", PATH_TO_LEFT_IMAGES)
PATH_TO_DISPARITY = os.path.join("data/", PATH_TO_DISPARITY)
PATH_TO_CAMERA = os.path.join("data/", PATH_TO_CAMERA)
PATH_TO_VEHICLE = os.path.join("data/", PATH_TO_VEHICLE)

print("Finished unzipping.")

--2020-03-25 12:39:13--  https://www.cityscapes-dataset.com/login/
Resolving www.cityscapes-dataset.com (www.cityscapes-dataset.com)... 139.19.217.8
Connecting to www.cityscapes-dataset.com (www.cityscapes-dataset.com)|139.19.217.8|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.cityscapes-dataset.com/downloads/ [following]
--2020-03-25 12:39:14--  https://www.cityscapes-dataset.com/downloads/
Reusing existing connection to www.cityscapes-dataset.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘index.html’

index.html              [ <=>                ]  42.77K  --.-KB/s    in 0.1s    

2020-03-25 12:39:16 (361 KB/s) - ‘index.html’ saved [43792]

--2020-03-25 12:39:18--  https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%2199734&authkey=AC5K24PFcrSPFl4
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)

We start with the detection of objects of interest. Faster R-CNN is used for this task. (Julius)

In [0]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']
static_classes = []              

Loading an on COCO pre-trained Faster R-CNN 


Installing Seamless Scene Segmentation from https://github.com/mapillary/seamseg
For Seamless Scene Segmentation a pre-trained version on the Mapillary Dataset exists. Dataloaders for Cityscapes seem to exist, too.

It is a network for panoptic segmentation basen on Mask-R-CNN. Alternatively the panoptic variant in the detectron2 reopisitory can also be trained on Cityscapes or Mapillary. Dataloaders seem to exist for both Datasets. 

In [3]:
#@title
!pip install git+https://github.com/mapillary/seamseg.git
#!pip install wget

url = 'https://drive.google.com/file/d/1ULhd_CZ24L8FnI9lZ2H6Xuf03n6NA_-Y/view'
!wget $url


  Cloning https://github.com/mapillary/seamseg.git to /tmp/pip-req-build-kpb8mbuu
  Running command git clone -q https://github.com/mapillary/seamseg.git /tmp/pip-req-build-kpb8mbuu
  Created wheel for seamseg: filename=seamseg-0.1.dev31+g18e28cc-cp36-cp36m-linux_x86_64.whl size=7004851 sha256=d6fd39949aade8d2596ae1f3a7c034f12f1f5149fc078ecd1861e1af1b1fea3f
  Stored in directory: /tmp/pip-ephem-wheel-cache-vutlpq_8/wheels/34/ae/9b/f7108b6552df4829b6952190c9af1ec04fc171251cdc5d9ea9
Successfully built seamseg
--2020-03-25 12:45:09--  https://drive.google.com/file/d/1ULhd_CZ24L8FnI9lZ2H6Xuf03n6NA_-Y/view
Resolving drive.google.com (drive.google.com)... 64.233.167.101, 64.233.167.113, 64.233.167.138, ...
Connecting to drive.google.com (drive.google.com)|64.233.167.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘view’

view                    [ <=>                ]  67.26K  --.-KB/s    in 0.03s   

2020-03-25 12:45:10 (2.29 M

These functions convert a difference in coordinates (lat., long.) to a difference in metric frame (x, y) and vice versa. Both conversions are dependent on the current latitude.
Under the assumption that we are in Europe, east of 0°!

Approximative conversions:

* Latitude: 1 deg = 110.574 km
* Longitude: 1 deg = 111.320*cos(latitude) km


In [0]:
# Felix 
# Helper functions for transformations.

import numpy as np


# Calculates the difference between two coordinates in meters (x to east, y to north).
# See below for example.
def coord_diff_to_metric_diff(d_lat, d_long, lat):
  d_y = d_lat * 110574
  d_x = d_long * 111320 * np.cos(np.radians(lat))
  return d_x, d_y


# Converts the metric difference to differences in latitude and longitude.
# See below for example.
def metric_diff_to_coord_diff(d_x, d_y, lat):
  d_lat = d_y / 110574
  d_long = d_x / 111320 / np.cos(np.radians(lat))
  return d_lat, d_long


# Returns the transformation matrix: world frame (-> vehicle frame) -> camera frame
# Usage: [x_world; 1] = camera_frame_to_world_transform(args).dot([x_camera_frame; 1])
# All angles in radians, distances in meters!
# See below for example.
def camera_frame_to_world_transform(heading, yaw_ext, pitch_ext, roll_ext, x_ext, y_ext, z_ext):
    # Coordinate axis switch for camera.
    C_c = np.array([[0., -1.,  0., 0.],
                    [0.,  0., -1., 0.],
                    [1.,  0.,  0., 0.],
                    [0.,  0.,  0., 1.]])
    # Vehicle to camera transformation matrix.
    c_y = np.cos(yaw_ext)
    c_p = np.cos(pitch_ext)
    c_r = np.cos(roll_ext)
    s_y = np.sin(yaw_ext)
    s_p = np.sin(pitch_ext)
    s_r = np.sin(roll_ext)
    T_v_c = np.array([[c_y*c_p, c_y*s_p*s_r-s_y*c_r, c_y*s_p*c_r+s_y*s_r, x_ext],
                      [s_y*c_p, s_y*s_p*s_r+c_y*c_r, s_y*s_p*c_r-c_y*s_r, y_ext],
                      [   -s_p,             c_p*s_r,             c_p*c_r, z_ext],
                      [     0.,                  0.,                  0.,    1.]])

    # World to vehicle transformation matrix.
    c_h = np.cos(heading)
    s_h = np.sin(heading)
    T_w_v = np.array([[c_h, -s_h, 0., 0.],
                      [s_h,  c_h, 0., 0.],
                      [ 0.,   0., 1., 0.],
                      [ 0.,   0., 0., 1.]])

    # Return camera space to vehicle transform.
    return T_w_v.dot(T_v_c.dot(np.linalg.inv(C_c)))

In [0]:
# Testing and example (taken from Johannes' map pic on WhatsApp):
# Converting coords into metrics:
d_x, d_y = coord_diff_to_metric_diff(48.370-48.369, 10.896-10.894, 48.368)
print("Delta x in meters: {}".format(d_x))
print("Delta y in meters: {}".format(d_y))

# Converting back to coords: should be the same:
d_lat, d_long = metric_diff_to_coord_diff(d_x, d_y, 48.368)
print("Difference in latitude: {}".format(d_lat))
print("Difference in longitude: {}".format(d_long))

# Convert a camera frame coordinate to world coordinate:
x_example = np.array([[8.1], [0.7], [0.7]])
x_world_example = camera_frame_to_world_transform(0.25, 0.0057, 0.055, 0.0, 1.7, -0.1, 1.3).dot(np.vstack((x_example, 1.0)))
print("Camera coordinate: ")
print(x_example)
print("World coordinate: ")
print(x_world_example)

In [0]:
# Loading results from json file stored in FEATURE_FILE 
FEATURE_FILE = "results.json"
with open(FEATURE_FILE) as json_file:
    features = json.load(json_file)

Plotting ground truth trajectory of vehicle. (Map image muss im Moment immer noch manuell gedownloaded und hinzugefügt werden)

In [0]:
# Loading ground truth data from json file and saving it as pandas array
import pandas as pd 
import os
import json
import glob
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt

DATA_DIR="data/vehicle/train_extra/schweinfurt"
images_path = list(glob.iglob(os.path.join(DATA_DIR, '*.*')))
n_images = len(images_path)
i = 0
# create space for long at lat
gpsLatitude = np.zeros(n_images)
gpsLongitude = np.zeros(n_images)

for image_path in images_path:
  # Load json from path
  with open(image_path) as json_file:
    data = json.load(json_file)
    gpsLatitude[i] = data['gpsLatitude']
    gpsLongitude[i] = data['gpsLongitude']
    i += 1
df = pd.DataFrame({'longitude': gpsLongitude,
'latitude': gpsLatitude})
BBox = ((df.longitude.min(),   df.longitude.max(),      
         df.latitude.min(), df.latitude.max()))

longdif = df.longitude.max() - df.longitude.min()
latgdif = df.latitude.max() - df.latitude.min()
aspectr = longdif/latgdif

image = plt.imread('/content/map (4).png')

fig, ax = plt.subplots(figsize = (8,7))
ax.scatter(df.longitude, df.latitude)
ax.set_title('Plotting Trajectiory')
ax.set_xlim(BBox[0],BBox[1])
ax.set_ylim(BBox[2],BBox[3])
ax.imshow(image, zorder=0, extent = BBox)
ax.set_aspect(aspect= aspectr)

plt.show()


In [0]:
DATA_DIR="data/vehicle/train_extra/schweinfurt"
images_path = list(glob.iglob(os.path.join(DATA_DIR, '*.*')))
n_images = len(images_path)
i = 0

# create space for long at lat
gpsLatitude_actual = np.zeros(n_images)
gpsLongitude_actual = np.zeros(n_images)
gpsHeading_actual = np.zeros(n_images)
gpsLatitude_delta = np.zeros(n_images)
gpsLongitude_delta = np.zeros(n_images)
gpsHeading_delta = np.zeros(n_images)


for image_path in images_path:
  # Load json from path
  with open(image_path) as json_file:
    data = json.load(json_file)
    gpsLatitude_actual[i] = data['gpsLatitude']
    gpsLongitude_actual[i] = data['gpsLongitude']
    gpsHeading_actual[i] = data['gpsHeading']
    if i > 0:
      gpsLatitude_delta[i] = gpsLatitude_actual[i] - gpsLatitude_actual[i-1]
      gpsLongitude_delta[i] = gpsLongitude_actual[i] - gpsLongitude_actual[i-1]
      gpsHeading_delta[i] = gpsHeading_actual[i] - gpsHeading_actual[i-1]
  i+=1

# Actual coordinates in x, y starting at 0, 0

gps_step_delta_X = np.zeros(n_images)
gps_total_delta_X = np.zeros(n_images)
gps_step_delta_Y = np.zeros(n_images)
gps_total_delta_Y = np.zeros(n_images)
gps_total_delta_Heading = np.zeros(n_images)

i = 0

for image_path in images_path:
  if i > 0:
    gps_step_delta_X[i], gps_step_delta_Y[i] = coord_diff_to_metric_diff(gpsLatitude_delta[i], gpsLongitude_delta[i], gpsLatitude_actual[i])
    gps_total_delta_X[i] = gps_total_delta_X[i-1] + gps_step_delta_X[i]
    gps_total_delta_Y[i] = gps_total_delta_Y[i-1] + gps_step_delta_Y[i]
    gps_total_delta_Heading[i] = gps_total_delta_Heading[i-1] + gpsHeading_delta[i]
  i+=1



In [0]:
# Get Camera Calibration Data

DATA_DIR="data/camera/train_extra/schweinfurt"
images_path = list(glob.iglob(os.path.join(DATA_DIR, '*.*')))
n_images = len(images_path)
i = 0

# create space for long at lat
roll  = np.zeros(n_images)
pitch = np.zeros(n_images)
yaw = np.zeros(n_images)
baseline = np.zeros(n_images)
offset_x = np.zeros(n_images)
offset_y = np.zeros(n_images)
offset_z = np.zeros(n_images)
f_x = np.zeros(n_images)
f_y = np.zeros(n_images)
u_0 = np.zeros(n_images)
v_0 = np.zeros(n_images)

for image_path in images_path:
  # Load json from path
  with open(image_path) as json_file:
    data = json.load(json_file)
    roll[i] = data['extrinsic']['roll']
    pitch[i] = data['extrinsic']['pitch']
    yaw[i] = data['extrinsic']['yaw']
    baseline[i] = data['extrinsic']['baseline']
    offset_x[i] = data['extrinsic']['x']
    offset_y[i] = data['extrinsic']['y']
    offset_z[i] = data['extrinsic']['z']
    f_x[i] = data['intrinsic']['fx']
    f_y[i] = data['intrinsic']['fy']
    u_0[i] = data['intrinsic']['u0']
    v_0[i] = data['intrinsic']['v0']
  i+=1



In [0]:
# Calculate real 3D position
def pixel_to_3Dposition_converter(x, depth, focal):
  x_3D = x * depth / focal
  return x_3D

# Calculate box center to get average disparity
def boundaries_to_box_center(x_1, x_2):
  x_center = (x_1 + x_2)/2
  return x_center

# Get camera focal length and baseline (To be corrected, pixels are NOT quadratic here fx =/= fy)
def camera_parameters(fx, fy):
  # Take average, which is Bullshit, but not better idea yet
  focal = (fx + fy)/2
  # Get baseline value from calibration data
  baseline = 0.222384
  return focal, baseline

# Get disparities of all points (pixels)
def disparity_catcher(x, y, i):
  DATA_DIR="data/disparity/train_extra/schweinfurt"
  im_directory = list(glob.iglob(os.path.join(DATA_DIR, '*.*')))
  n_im = len(im_directory)
  l = 0
  disp = 0

  for im_number in im_directory:
  # Load image from path
    img = cv2.imread(im_number, cv2.IMREAD_UNCHANGED)
    if l == i:
      # Changed to read precise disparity. (Felix)
      # x and y values were mixed up! Please take care next time.
      #if disp != 0:
      disp = (img[y, x] - 1.) / 256.

     # print(disp)
      return disp
    else:
      l += 1

  # return disp

# Get disparity of center point
# def box_center_disparity(x_center, y_center):
  # disparity_box_center = pix_val_flat[1024 * 2048 * 4 - 2048 * y_center * 4 + x_center * 4]
  # return disparity_box_center

# Convert disparity to actual depth information
def disparity_to_depth(disparity_box_center):
  if disparity_box_center > 0:
    depth_center = baseline * focal / disparity_box_center
  else:
    depth_center = 100
  return depth_center

# Transformation to world frame coordinates
# _3dImage	=	cv.reprojectImageTo3D('data/disparity/train_extra/schweinfurt/schweinfurt_000000_000000_disparity.png', Q)


In [0]:
import math
import csv
import pandas as pd
import cv2
import glob

# Get focal length from calibration data
fx = 2273.82
fy = 2236.9989965173254
# baseline = 0.222384

focal, baseline = camera_parameters(fx, fy)


DATA_DIR="data/Numerical_Results"
results_path = list(glob.iglob(os.path.join(DATA_DIR, '*.*')))
n_images = len(images_path)
k = 0 # Images
t = 0 # Total object number

p = 0
num_objects = 0

for x in results_path:
  # Load json from path
  with open(x) as json_file:
    data = json.load(json_file)
    for i in images_path:
      num_objects = num_objects + int(len(data['results'][k]['classes']))
      p += 1

u1 = np.zeros(num_objects)
v1 = np.zeros(num_objects)
u2 = np.zeros(num_objects)
v2 = np.zeros(num_objects)
x1 = np.zeros(num_objects)
y1 = np.zeros(num_objects)
x2 = np.zeros(num_objects)
y2 = np.zeros(num_objects)
x_3D_rel_car = np.zeros(num_objects)
y_3D_rel_car = np.zeros(num_objects)
z_3D_rel_car = np.zeros(num_objects)
obj_vector = np.zeros(shape=(num_objects,4))

for x in results_path:
  # Load json from path
  with open(x) as json_file:
    data = json.load(json_file)
    for i in images_path:
      print("Processing image number", k)
      n_objects = int(len(data['results'][k]['classes']))
      #print(n_objects)
      #print(data['results'][k]['boxes'][0][1])
      for o in range(n_objects):

        # Get coordinates in image frame
        u1[t] = data['results'][k]['boxes'][o][0]
        v1[t] = data['results'][k]['boxes'][o][1]
        u2[t] = data['results'][k]['boxes'][o][2]
        v2[t] = data['results'][k]['boxes'][o][3]

        # Calculate center point in image frame
        u_cent = int(boundaries_to_box_center(u1[t], u2[t]))
        v_cent = int(boundaries_to_box_center(v1[t], v2[t]))

        # Get disparity of center point in image frame
        disp_cent = disparity_catcher(u_cent, v_cent, k)

        # Calculate depth from disparity
        depth_cent = disparity_to_depth(disp_cent)

        # Transform coordinates from image plane to camera frame
        x1[t] = depth_cent * (u1[t] - u_0[k]) / f_x[k]
        y1[t] = depth_cent * (v1[t] - v_0[k]) / f_y[k]
        x2[t] = depth_cent * (u2[t] - u_0[k]) / f_x[k]
        y2[t] = depth_cent * (v2[t] - v_0[k]) / f_y[k]

        # Calculate box center in camera frame
        x_cent = boundaries_to_box_center(x1[t], x2[t])
        y_cent = boundaries_to_box_center(y1[t], y2[t])

        # Save object coordinates in camera frame
        x_3D_rel_car[t] = x_cent
        y_3D_rel_car[t] = y_cent
        z_3D_rel_car[t] = depth_cent

        # Transform object coordinates to world frame
        input_vector = np.array([[x_3D_rel_car[t]], [y_3D_rel_car[t]], [z_3D_rel_car[t]]])
        output_vector = camera_frame_to_world_transform(gps_total_delta_Heading[k], yaw[k], pitch[k], roll[k], offset_x[k], offset_y[k], offset_z[k]).dot(np.vstack((input_vector, 1.0)))
        
        # Transform back to Lat / Long and save to vector
        d_lat, d_long = metric_diff_to_coord_diff(output_vector[0], output_vector[1], gpsLatitude_actual[k])
        obj_vector[t][0] = gpsLatitude_actual[k] + d_lat
        obj_vector[t][1] = gpsLongitude_actual[k] + d_long
        obj_vector[t][2] = output_vector[2]
        obj_vector[t][3] = data['results'][k]['classes'][o]
        t += 1
      k += 1

      # if k == N loop incl. break to be removed, only for debugging purposes
      if k == 100:
        ## Write to file

        # Create DataFrame 
        # mapping_output_df = pd.DataFrame({'Longitude': obj_vector[:, 1], 'Latitude': obj_vector[:, 0], 'z': obj_vector[:, 2], 'Semantic Label': obj_vector[:, 3]})

        # Print the output. 
        # print(mapping_output_df) 

        ##
        break


In [0]:
### Write to DataFrame and to csv file

# Create DataFrame 
mapping_output_df = pd.DataFrame({'Longitude': obj_vector[:, 1], 'Latitude': obj_vector[:, 0], 'z': obj_vector[:, 2], 'Semantic Label': obj_vector[:, 3]})

# Print the output
print(mapping_output_df)

# Write output to csv file
mapping_output_df.to_csv(r'data/mapping_output.csv', index = False)